### Prepare session for time alignment

In [17]:
from harp.clock import decode_harp_clock
from beh_functions import session_dirs, parseSessionID
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import os
import numpy as np

In [12]:
session_id = 'behavior_716325_2024-05-31_10-31-14'
session_dir = session_dirs(session_id)

In [15]:
session = Session(session_dir['session_dir'])
recording = session.recordnodes[0].recordings[0]
harp_line, nidaq_stream_name, source_node_id, figure = search_harp_line(recording, session_dir['session_dir'])
figure.savefig(os.path.join(session_dir['alignment_dir'], 'harp_line.png'))
print(F'Harp line {harp_line} found in {session_id}')

Harp line [4] found in behavior_716325_2024-05-31_10-31-14


In [19]:
events = recording.events
harp_events = events[
    (events.stream_name == nidaq_stream_name)
    & (events.processor_id == source_node_id)
    & (events.line == harp_line[0])
]

harp_states = harp_events.state.values
harp_timestamps_local = harp_events.timestamp.values
local_times, harp_times = decode_harp_clock(
    harp_timestamps_local, harp_states
)
np.save(os.path.join(session_dir['alignment_dir'], 'harp_times.npy'), harp_times)
np.save(os.path.join(session_dir['alignment_dir'], 'local_times.npy'), local_times)

print('Harp times saved to: {}'.format(os.path.join(session_dir['alignment_dir'], 'harp_times.npy')))
print('Local times saved to: {}'.format(os.path.join(session_dir['alignment_dir'], 'local_times.npy')))

Harp times saved to: /root/capsule/scratch/behavior_716325_2024-05-31_10-31-14/alignment/harp_times.npy
Local times saved to: /root/capsule/scratch/behavior_716325_2024-05-31_10-31-14/alignment/local_times.npy
